In [1]:
import pandas as pd
import numpy as np
import random

In [2]:
df = pd.read_csv('Nifty 50 Historical Data.csv')
print(len(df), df.columns)

350 Index(['Date', 'Price', 'Open', 'High', 'Low', 'Vol.', 'Change %'], dtype='object')


In [3]:
lst_equity_historic_monthly_returns = []
for row_idx, row in df.iterrows():
    lst_equity_historic_monthly_returns.append(float(row['Change %'].strip('%')))

In [4]:
len(lst_equity_historic_monthly_returns)
print(lst_equity_historic_monthly_returns[:5])
print(lst_equity_historic_monthly_returns[-5:])

[-2.34, -2.02, -0.31, -6.22, 2.28]
[13.1, -0.73, 16.98, -6.62, 5.39]


In [5]:
lst_equity_historic_monthly_returns.reverse()
print(lst_equity_historic_monthly_returns[:5])
print(lst_equity_historic_monthly_returns[-5:])

[5.39, -6.62, 16.98, -0.73, 13.1]
[2.28, -6.22, -0.31, -2.02, -2.34]


In [6]:
df = pd.read_csv('CPIndex_Jan14-To-Dec24.csv')
print(len(df), df.columns)

130 Index(['Year', 'Month', 'State', 'Group', 'Sub Group', 'Description', 'Rural',
       'Urban', 'Combined', 'Status'],
      dtype='object')


In [7]:
lst_inflation_historic_monthly_YoY = []
for row_idx, row in df.iterrows():
    lst_inflation_historic_monthly_YoY.append(float(row['Combined']))

In [8]:
len(lst_inflation_historic_monthly_YoY)
print(lst_inflation_historic_monthly_YoY[:5])
print(lst_inflation_historic_monthly_YoY[-5:])

[8.6, 7.88, 8.25, 8.48, 8.33]
[3.65, 5.49, 6.21, 5.48, 5.22]


**Parameters**

In [9]:
Y = 30 #No. of years to simulate
W = 3.5 #Withdrawal rate
DR_diff = 1.0 #Debt returns differential with current inflation

IC = 20000000 # Initial Corpus
M = 100 # No. of months in a cycle of equity-debt shuffling (needed in "hybrid" strategy)
Debt_ratio = 0.4 # Fraction of initial corpus in debt (needed in "fixed hybrid" strategy)
Rb_M = 12 # No. of months to check and re-balance debt/equity ratio to Debt_ratio (needed in "fixed hybrid" strategy)

In [10]:
random.randint(0,len(lst_equity_historic_monthly_returns)-12)

305

In [11]:
def generate_random_samples_for_equity_returns(lst_equity_historic_monthly_returns, Y, chunk_size=12):
    sampled_equity_returns = []
    num_chunks = Y*12 // chunk_size
    #print(num_chunks)
    random.seed()
    for i in range(num_chunks):
        rand_idx = random.randint(0,len(lst_equity_historic_monthly_returns)-chunk_size)
        #print(rand_idx)
        for j in range(chunk_size):
            sampled_equity_returns.append(lst_equity_historic_monthly_returns[rand_idx+j])
    #print(len(sampled_equity_returns), Y*12)
    rand_idx = random.randint(0,len(lst_equity_historic_monthly_returns)-(Y*12-len(sampled_equity_returns)))
    for j in range(Y*12-len(sampled_equity_returns)):
        sampled_equity_returns.append(lst_equity_historic_monthly_returns[rand_idx+j])
    #print(len(sampled_equity_returns), Y*12)
    return sampled_equity_returns

In [12]:
sampled_equity_returns = generate_random_samples_for_equity_returns(lst_equity_historic_monthly_returns, Y)
print(np.mean(sampled_equity_returns), len(sampled_equity_returns))

0.5291666666666666 360


In [13]:
def generate_random_samples_for_inflation(lst_inflation_historic_monthly_YoY, Y, chunk_size=12):
    sampled_inflation_rates = []
    num_chunks = Y*12 // chunk_size
    #print(num_chunks)
    random.seed()
    for i in range(num_chunks):
        rand_idx = random.randint(0,len(lst_inflation_historic_monthly_YoY)-chunk_size)
        #print(rand_idx)
        for j in range(chunk_size):
            sampled_inflation_rates.append(lst_inflation_historic_monthly_YoY[rand_idx+j])
    #print(len(sampled_inflation_rates), Y*12)
    rand_idx = random.randint(0,len(lst_inflation_historic_monthly_YoY)-(Y*12-len(sampled_inflation_rates)))
    for j in range(Y*12-len(sampled_inflation_rates)):
        sampled_inflation_rates.append(lst_inflation_historic_monthly_YoY[rand_idx+j])
    #print(len(sampled_inflation_rates), Y*12)
    return sampled_inflation_rates

In [14]:
sampled_inflation_rates = generate_random_samples_for_inflation(lst_inflation_historic_monthly_YoY, Y)
print(np.mean(sampled_inflation_rates), len(sampled_inflation_rates))

4.795333333333334 360


In [15]:
def run_simulation(Y, W, IC, DR_diff, M, sampled_equity_returns, sampled_inflation_rates, verbose=False, write_to_file=None):
    month_no = []; withdrawl_amount = []; debt_corpus = []; equity_corpus = []

    first_month_withdrawl_amount = round((IC * W/100.0) / 12.0,0)
    initial_debt_corpus = M * first_month_withdrawl_amount
    initial_equity_corpus = IC - initial_debt_corpus
    if(verbose):
        print('first_month_withdrawl_amount:', first_month_withdrawl_amount)
        print('initial_debt_corpus:', initial_debt_corpus)
        print('initial_equity_corpus:', initial_equity_corpus)

    #For the first month
    withdrawl_amount.append(first_month_withdrawl_amount)
    debt_corpus.append(initial_debt_corpus-first_month_withdrawl_amount)
    equity_corpus.append(initial_equity_corpus)
    if(verbose):
        print(1, '\t', withdrawl_amount[-1], '\t', debt_corpus[-1], '\t', equity_corpus[-1])
    month_no.append(1)
    for i in range(1, M):
        new_equity_corpus = round(equity_corpus[-1] * (1+sampled_equity_returns[i]/100.0),0)
        new_withdrawl_amount = round(withdrawl_amount[-1] * (1+(sampled_inflation_rates[i]/12.0)/100.0),0)
        new_debt_corpus = round(debt_corpus[-1] * (1+((sampled_inflation_rates[i]+DR_diff)/12.0)/100.0),0)
        new_debt_corpus = new_debt_corpus - new_withdrawl_amount
        
        withdrawl_amount.append(new_withdrawl_amount)
        debt_corpus.append(new_debt_corpus)
        equity_corpus.append(new_equity_corpus)
        if(verbose):
            print(i+1, '\t', withdrawl_amount[-1], '\t', debt_corpus[-1], '\t', equity_corpus[-1], '\t', sampled_equity_returns[i])
        month_no.append(i+1)
    r = 1
    while(True):
        #Rebalancing after M months
        new_equity_corpus = round(equity_corpus[-1] * (1+sampled_equity_returns[r*M]/100.0),0)
        new_withdrawl_amount = round(withdrawl_amount[-1] * (1+(sampled_inflation_rates[r*M]/12.0)/100.0),0)
        new_debt_corpus = round(debt_corpus[-1] * (1+((sampled_inflation_rates[r*M]+DR_diff)/12.0)/100.0),0)
        new_debt_corpus_ideal = M * new_withdrawl_amount
        new_equity_corpus = new_equity_corpus - (new_debt_corpus_ideal - new_debt_corpus)
        new_debt_corpus = new_debt_corpus_ideal - new_withdrawl_amount
        withdrawl_amount.append(new_withdrawl_amount)
        debt_corpus.append(new_debt_corpus)
        equity_corpus.append(new_equity_corpus)
        if(verbose):
            print('\n', r*M+1, '\t', withdrawl_amount[-1], '\t', debt_corpus[-1], '\t', equity_corpus[-1], '**Re-balancing month**\n')
        month_no.append(r*M+1)
        
        flag = False
        for i in range(1, M):
            if(r*M+i >= len(sampled_equity_returns)):
                flag = True
                break
            new_equity_corpus = round(equity_corpus[-1] * (1+sampled_equity_returns[r*M+i]/100.0),0)
            new_withdrawl_amount = round(withdrawl_amount[-1] * (1+(sampled_inflation_rates[r*M+i]/12.0)/100.0),0)
            new_debt_corpus = round(debt_corpus[-1] * (1+((sampled_inflation_rates[r*M+i]+DR_diff)/12.0)/100.0),0)
            new_debt_corpus = new_debt_corpus - new_withdrawl_amount
            
            withdrawl_amount.append(new_withdrawl_amount)
            debt_corpus.append(new_debt_corpus)
            equity_corpus.append(new_equity_corpus)
            if(verbose):
                print(r*M+i+1, '\t', withdrawl_amount[-1], '\t', debt_corpus[-1], '\t', equity_corpus[-1], '\t', sampled_equity_returns[i])
            month_no.append(r*M+i+1)
        if(flag):
            break
        r = r + 1
    
    if(write_to_file):
        df = pd.DataFrame()
        print(len(month_no), len(withdrawl_amount), len(debt_corpus), len(equity_corpus), 
              len(sampled_equity_returns), len(sampled_inflation_rates))
        df['month'] = month_no
        df['withdrawl_amount'] = withdrawl_amount
        df['debt_corpus'] = debt_corpus
        df['equity_corpus'] = equity_corpus
        df['sampled_equity_returns'] = sampled_equity_returns
        df['sampled_inflation_rates'] = sampled_inflation_rates
        df.to_excel(write_to_file, index=False)
    return (month_no, withdrawl_amount, debt_corpus, equity_corpus)

In [16]:
print(np.mean(sampled_equity_returns[:100]))

1.4804


In [17]:
run_simulation(Y, W, IC, DR_diff, M, sampled_equity_returns, sampled_inflation_rates, write_to_file='temp_simulation_run_3.xlsx')
#_,_,_,_ = run_simulation(Y, W, IC, DR_diff, M, sampled_equity_returns, sampled_inflation_rates)

360 360 360 360 360 360


([1,
  2,
  3,
  4,
  5,
  6,
  7,
  8,
  9,
  10,
  11,
  12,
  13,
  14,
  15,
  16,
  17,
  18,
  19,
  20,
  21,
  22,
  23,
  24,
  25,
  26,
  27,
  28,
  29,
  30,
  31,
  32,
  33,
  34,
  35,
  36,
  37,
  38,
  39,
  40,
  41,
  42,
  43,
  44,
  45,
  46,
  47,
  48,
  49,
  50,
  51,
  52,
  53,
  54,
  55,
  56,
  57,
  58,
  59,
  60,
  61,
  62,
  63,
  64,
  65,
  66,
  67,
  68,
  69,
  70,
  71,
  72,
  73,
  74,
  75,
  76,
  77,
  78,
  79,
  80,
  81,
  82,
  83,
  84,
  85,
  86,
  87,
  88,
  89,
  90,
  91,
  92,
  93,
  94,
  95,
  96,
  97,
  98,
  99,
  100,
  101,
  102,
  103,
  104,
  105,
  106,
  107,
  108,
  109,
  110,
  111,
  112,
  113,
  114,
  115,
  116,
  117,
  118,
  119,
  120,
  121,
  122,
  123,
  124,
  125,
  126,
  127,
  128,
  129,
  130,
  131,
  132,
  133,
  134,
  135,
  136,
  137,
  138,
  139,
  140,
  141,
  142,
  143,
  144,
  145,
  146,
  147,
  148,
  149,
  150,
  151,
  152,
  153,
  154,
  155,
  156,
  157,
  158,
  

**Running the simulation for multiple runs**

In [18]:
success = 0
RUNS = 10000
for run in range(RUNS):
    sampled_equity_returns = generate_random_samples_for_equity_returns(lst_equity_historic_monthly_returns, Y)
    sampled_inflation_rates = generate_random_samples_for_inflation(lst_inflation_historic_monthly_YoY, Y)
    _,_,_,equity_corpus = run_simulation(Y, W, IC, DR_diff, M, sampled_equity_returns, sampled_inflation_rates)
    if(equity_corpus[-1] > 0):
        success = success + 1

In [19]:
print(success*100.0/RUNS)

83.12


**Simulation for pure equity withdrawal strategy**

In [20]:
def run_simulation_pure_equity(Y, W, IC, sampled_equity_returns, sampled_inflation_rates, verbose=False, write_to_file=None):
    month_no = []; withdrawl_amount = []; equity_corpus = []

    first_month_withdrawl_amount = round((IC * W/100.0) / 12.0,0)
    initial_equity_corpus = IC
    if(verbose):
        print('first_month_withdrawl_amount:', first_month_withdrawl_amount)
        print('initial_equity_corpus:', initial_equity_corpus)

    #For the first month
    withdrawl_amount.append(first_month_withdrawl_amount)
    equity_corpus.append(initial_equity_corpus - first_month_withdrawl_amount)
    if(verbose):
        print(1, '\t', withdrawl_amount[-1], '\t', equity_corpus[-1])
    month_no.append(1)
    for i in range(1, len(sampled_equity_returns)):
        new_equity_corpus = round(equity_corpus[-1] * (1+sampled_equity_returns[i]/100.0),0)
        new_withdrawl_amount = round(withdrawl_amount[-1] * (1+(sampled_inflation_rates[i]/12.0)/100.0),0)
        new_equity_corpus = new_equity_corpus - new_withdrawl_amount
        
        withdrawl_amount.append(new_withdrawl_amount)
        equity_corpus.append(new_equity_corpus)
        if(verbose):
            print(i+1, '\t', withdrawl_amount[-1], '\t', equity_corpus[-1], '\t', sampled_equity_returns[i])
        month_no.append(i+1)
    
    if(write_to_file):
        df = pd.DataFrame()
        print(len(month_no), len(withdrawl_amount), len(equity_corpus), 
              len(sampled_equity_returns), len(sampled_inflation_rates))
        df['month'] = month_no
        df['withdrawl_amount'] = withdrawl_amount
        df['equity_corpus'] = equity_corpus
        df['sampled_equity_returns'] = sampled_equity_returns
        df['sampled_inflation_rates'] = sampled_inflation_rates
        df.to_excel(write_to_file, index=False)
    return (month_no, withdrawl_amount, equity_corpus)

In [21]:
_,_,_=run_simulation_pure_equity(Y, W, IC, sampled_equity_returns, sampled_inflation_rates, verbose=False, write_to_file='temp_simulation_run_equity_only.xlsx')

360 360 360 360 360


**Simulation for hybrid debt-equity strategy**

In [22]:
def run_simulation_fixed_hybrid(Y, W, IC, DR_diff, Debt_ratio, Rb_M, sampled_equity_returns, sampled_inflation_rates, verbose=False, write_to_file=None):
    month_no = []; withdrawl_amount = []; debt_corpus = []; equity_corpus = []

    first_month_withdrawl_amount = round((IC * W/100.0) / 12.0,0)
    initial_debt_corpus = round(Debt_ratio*IC,0)
    initial_equity_corpus = IC - initial_debt_corpus
    if(verbose):
        print('first_month_withdrawl_amount:', first_month_withdrawl_amount)
        print('initial_debt_corpus:', initial_debt_corpus)
        print('initial_equity_corpus:', initial_equity_corpus)

    #For the first month
    withdrawl_amount.append(first_month_withdrawl_amount)
    debt_corpus.append(initial_debt_corpus-round(Debt_ratio*first_month_withdrawl_amount,0))
    equity_corpus.append(initial_equity_corpus-round((1-Debt_ratio)*first_month_withdrawl_amount,0))
    if(verbose):
        print(1, '\t', withdrawl_amount[-1], '\t', debt_corpus[-1], '\t', equity_corpus[-1])
    month_no.append(1)
    for i in range(1, len(sampled_equity_returns)):
        new_equity_corpus = round(equity_corpus[-1] * (1+sampled_equity_returns[i]/100.0),0)
        new_withdrawl_amount = round(withdrawl_amount[-1] * (1+(sampled_inflation_rates[i]/12.0)/100.0),0)
        new_debt_corpus = round(debt_corpus[-1] * (1+((sampled_inflation_rates[i]+DR_diff)/12.0)/100.0),0)

        if(Rb_M > 0 and i%Rb_M == 0):
            #print('Rebalancing.. (before)', new_debt_corpus, new_equity_corpus)
            total_corpus = new_debt_corpus + new_equity_corpus
            new_debt_corpus = round(Debt_ratio*total_corpus,0)
            new_equity_corpus = round((1-Debt_ratio)*total_corpus,0)
            #print('(after)', new_debt_corpus, new_equity_corpus)

        curr_debt_ratio = new_debt_corpus / (new_debt_corpus + new_equity_corpus)
        curr_equity_ratio = 1.0 - curr_debt_ratio
        
        #new_debt_corpus = new_debt_corpus - round(Debt_ratio*new_withdrawl_amount,0)
        #new_equity_corpus = new_equity_corpus - round((1-Debt_ratio)*new_withdrawl_amount,0)
        new_debt_corpus = new_debt_corpus - round(curr_debt_ratio*new_withdrawl_amount,0)
        new_equity_corpus = new_equity_corpus - round(curr_equity_ratio*new_withdrawl_amount,0)
        
        withdrawl_amount.append(new_withdrawl_amount)
        debt_corpus.append(new_debt_corpus)
        equity_corpus.append(new_equity_corpus)
        if(verbose):
            print(i+1, '\t', withdrawl_amount[-1], '\t', debt_corpus[-1], '\t', equity_corpus[-1], '\t', sampled_equity_returns[i])
        month_no.append(i+1)
    
    if(write_to_file):
        df = pd.DataFrame()
        print(len(month_no), len(withdrawl_amount), len(debt_corpus), len(equity_corpus), 
              len(sampled_equity_returns), len(sampled_inflation_rates))
        df['month'] = month_no
        df['withdrawl_amount'] = withdrawl_amount
        df['debt_corpus'] = debt_corpus
        df['equity_corpus'] = equity_corpus
        df['sampled_equity_returns'] = sampled_equity_returns
        df['sampled_inflation_rates'] = sampled_inflation_rates
        df.to_excel(write_to_file, index=False)
    return (month_no, withdrawl_amount, debt_corpus, equity_corpus)

In [23]:
_,_,_,_=run_simulation_fixed_hybrid(Y, W, IC, DR_diff, Debt_ratio, Rb_M, sampled_equity_returns, sampled_inflation_rates, verbose=False, write_to_file='temp_simulation_run_fixed_hybrid.xlsx')

360 360 360 360 360 360


In [24]:
failure_pure_equity = 0
failure_fixed_hybrid = 0
failure_hybrid = 0
sample_file_written_pure_equity = False
sample_file_written_hybrid = False
RUNS = 10000
for run in range(RUNS):
    sampled_equity_returns = generate_random_samples_for_equity_returns(lst_equity_historic_monthly_returns, Y)
    sampled_inflation_rates = generate_random_samples_for_inflation(lst_inflation_historic_monthly_YoY, Y)
    
    _,_,equity_corpus_1 = run_simulation_pure_equity(Y, W, IC, sampled_equity_returns, sampled_inflation_rates)
    #print('1', equity_corpus_1[-1])
    if(equity_corpus_1[-1] < 0):
        failure_pure_equity = failure_pure_equity + 1
        #if(not(sample_file_written_pure_equity)):
        #    run_simulation_pure_equity(Y, W, IC, sampled_equity_returns, sampled_inflation_rates, verbose=False, write_to_file='simulation_run_equity_only_failure_example.xlsx')
        #    sample_file_written_pure_equity = True

    _,_,debt_corpus_2,equity_corpus_2 = run_simulation_fixed_hybrid(Y, W, IC, DR_diff, Debt_ratio, Rb_M, sampled_equity_returns, sampled_inflation_rates)
    #print('2', equity_corpus_2[-1])
    if(equity_corpus_2[-1] < 0 or debt_corpus_2[-1] < 0):
        failure_fixed_hybrid = failure_fixed_hybrid + 1
        #if(not(sample_file_written_pure_equity)):
        #    run_simulation_pure_equity(Y, W, IC, sampled_equity_returns, sampled_inflation_rates, verbose=False, write_to_file='simulation_run_equity_only_failure_example.xlsx')
        #    sample_file_written_pure_equity = True
    
    _,_,_,equity_corpus_3 = run_simulation(Y, W, IC, DR_diff, M, sampled_equity_returns, sampled_inflation_rates)
    if(equity_corpus_3[-1] < 0):
        failure_hybrid = failure_hybrid + 1
        #if(not(sample_file_written_hybrid)):
        #    run_simulation(Y, W, IC, DR_diff, M, sampled_equity_returns, sampled_inflation_rates, write_to_file='simulation_run_hybrid_failure_example.xlsx')
        #    sample_file_written_hybrid = True

    #if(equity_corpus_1[-1] < 0 and equity_corpus_2[-1] > 0):
    #    run_simulation_pure_equity(Y, W, IC, sampled_equity_returns, sampled_inflation_rates, verbose=False, write_to_file='simulation_run_equity_only_failure_when_hybrid_successful.xlsx')
    #    run_simulation(Y, W, IC, DR_diff, M, sampled_equity_returns, sampled_inflation_rates, write_to_file='simulation_run_hybrid_success_when_equity_only_fails.xlsx')
    
print('failure_pure_equity:', failure_pure_equity*100.0/RUNS)
print('failure_fixed_hybrid:', failure_fixed_hybrid*100.0/RUNS)
print('failure_hybrid:', failure_hybrid*100.0/RUNS)        

failure_pure_equity: 13.37
failure_fixed_hybrid: 7.15
failure_hybrid: 16.84
